# [作者 TsukiSama9292 - Github 連結](https://github.com/tsukisama9292)
## [參考文獻 - 連結](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# 依賴安裝

In [ ]:
### 在自己的電腦中才需要安裝 ffmpeg ， Colab 已經有安裝好 ffmpeg

# Windows : https://vocus.cc/article/64701a2cfd897800014daed0

# macOS : https://blog.csdn.net/csdn_yudong/article/details/129753111

# Linux，直接執行下方 sudo 指令
# !sudo apt update
# !sudo apt install ffmpeg

In [ ]:
%pip install -q yt-dlp
%pip install -q -U gradio

In [ ]:
import yt_dlp
import threading
import queue
import os
import zipfile
import shutil
from datetime import datetime
import gradio as gr
# 全局變量或全局狀態，用於保存播放清單信息
playlist_info = {
    'total_videos': 0,
    'downloaded_videos': 0,
}
count = True
ydl_opts = None  # 初始化 ydl_opts
### 616 是 1080p-Premium 通道
resolution_option={
'自動':['bestvideo'],
'8K(4320)':['571'],
'4K(2160)':['628','315','625','313','401'],
'2K(1440)':['623','308','620','271','400'],
'FHD(1080)':['312','617','299','303','270','614','137','399','248'],
'HD(720)':['298','612','302','136','609','398','247','311','232'],
'480':['231','606','135','244','397'],
'360':['230','605','18','134','243','396'],
'240':['604','229','133','242','395'],
'144':['603','269','278','160','394'],
}

# 最佳畫質
resolution_high={
'自動':['bestvideo'],
'8K(4320)':['571'],
'4K(2160)':['628','315','625','313','401'],
'2K(1440)':['623','308','620','271','400'],
'FHD(1080)':['312','617','299','303','270','614','137','399','248'],
'HD(720)':['298','612','302','136','609','398','247','311','232'],
'480':['231','606','135','244','397'],
'360':['230','605','18','134','243','396'],
'240':['604','229','133','242','395'],
'144':['603','269','278','160','394'],
}

# 平均
resolution_avg={
'自動':['bestvideo'],
'8K(4320)':['571'],
'4K(2160)':['315','313','401','628','625'],
'2K(1440)':['308','271','400','623','620'],
'FHD(1080)':['299','617','303','137','614','399','248','312','270'],
'HD(720)':['311','612','298','302','232','609','136','398','247'],
'480':['135','606','244','397','231'],
'360':['18','134','605','243','396','230'],
'240':['133','299','242','395','604'],
'144':['278','269','160','394','603'],
}

# 最小檔案
resolution_low={
'自動':['bestvideo'],
'8K(4320)': ['571'],
'4K(2160)': ['401', '313', '625', '315', '628'],
'2K(1440)': ['400', '271', '620', '308', '623'],
'FHD(1080)': ['248', '399', '137', '614', '270', '303', '299', '617', '312'],
'HD(720)': ['247', '398', '136', '609', '232', '302', '298', '612', '311'],
'480': ['397', '244', '135', '606', '231'],
'360': ['396', '243', '134', '18', '605', '230'],
'240': ['395', '242', '133', '229', '604'],
'144': ['394', '160', '278', '269', '603'],
}

def get_playlist_video_urls(YOUTUBE_URL):
    # 初始化yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    
    # 獲取播放清單的信息
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    
    # 檢查是否為播放清單
    if 'entries' in info:
        # 提取所有影片的 YouTube 連結
        video_links = [entry['webpage_url'] for entry in info['entries'] if 'webpage_url' in entry]
        return video_links
    else: # 只有一個影片直接回傳
        return [YOUTUBE_URL]

def get_video_quantity(YOUTUBE_URL):
    # 初始化yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    # 獲取播放清單的信息
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    if 'entries' in info:
        playlist_info['total_videos'] = len(info['entries'])
    else:
        playlist_info['total_videos'] = 1

def get_video_info(youtube_url):
    ydl = yt_dlp.YoutubeDL()
    
    # 獲取影片的詳細信息
    info = ydl.extract_info(youtube_url, download=False)
    
    # 提取標題、上傳者和上傳時間
    title = info.get('title', '未知標題')
    uploader = info.get('uploader', '未知')
    upload_date = info.get('upload_date', '未知')  # 上傳時間格式為 YYYYMMDD

    # 如果需要格式化上傳日期
    if upload_date != '未知':
        year = upload_date[:4]
        month = upload_date[4:6]
        day = upload_date[6:]
        formatted_upload_date = f"{year}-{month}-{day}"
    else:
        formatted_upload_date = '未知'
    # 獲取當前的日期和時間
    now = datetime.now()
    
    # 格式化為所需的字符串
    current_time = now.strftime("%Y-%m-%d %H:%M")
    
    return title, uploader, formatted_upload_date , current_time


# 定義進度回調函數
def progress_hook(q):
    def hook(d):
        # 更新播放清單的下載進度
        global playlist_info
        global count
        if d['status'] == 'downloading':
            speed = d.get('speed', 0)
            speed_mb = speed / 1024 / 1024 if speed is not None else 0
            q.put({
                '狀態': '下載中',
                '已下載大小(MB)': d['downloaded_bytes'] / 1024 / 1024,
                '當前影片總容量(MB)': d.get('total_bytes', 0) / 1024 / 1024,
                '網路速度(MB/秒)': speed_mb,
                '預設剩餘時間(秒)': d.get('eta', 0),
                '完成進度': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
        elif d['status'] == 'finished':
            if(count):
                count = False
            else:
                count = True
                playlist_info['downloaded_videos'] += 1
            q.put({
                '狀態': '單項下載完畢',
                '存放路徑': d['filename'],
                '完成進度': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
    return hook


# 定義下載函數
def download_Run(ydl_opts, url, q):
    ydl_opts['progress_hooks'] = [progress_hook(q)]
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def download_Thread(ydl_opts,url):
    # 創建一個 Queue 來傳遞進度消息
    q = queue.Queue()

    # 在獨立的執行緒中執行下載函數
    thread = threading.Thread(target=download_Run, args=(ydl_opts, url, q))
    thread.start()

    # 即時顯示下載進度
    while thread.is_alive() or not q.empty():
        try:
            progress = q.get(timeout=1)
            yield progress
        except queue.Empty:
            pass
        
def re_ydl_opts_resolution(YOUTUBE_URL,resolution,out_path):
    bool_list = [False] * len(resolution)
    global ydl_opts
    # 初始化yt-dlp
    title, uploader, formatted_upload_date , current_time = get_video_info(YOUTUBE_URL)
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': out_path,
        'postprocessors': [
            {
                'key': 'FFmpegMetadata'
            }
        ]
    }
    # 檢查畫質格式是否可用
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    formats = info.get('formats', [])
    for format in formats:
        for index,resolution_id in enumerate(resolution):  
            if format.get('format_id') == resolution_id:
                bool_list[index]=True
    
    for index,bool_i in enumerate(bool_list):
        if(bool_i): 
            ydl_opts = {
            'format': f'{resolution[index]}+bestaudio/best',
            'outtmpl': out_path,
            'postprocessors': [
                {
                    'key': 'FFmpegMetadata'
                }
            ]
            }
            break

def download_video(YOUTUBE_URL, do_zip,resolution):
    global ydl_opts
    if(do_zip):
        out_path='Downloads_TMP/%(title)s.%(ext)s'
        links = get_playlist_video_urls(YOUTUBE_URL)
        for link in links:
            re_ydl_opts_resolution(link,resolution,out_path)
            yield from download_Thread(ydl_opts,link)
        if has_files('Downloads_TMP'):
            zip_files('Video')
            yield "影片下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取影片"
    else:
        out_path='Downloads/Video/%(title)s.%(ext)s'
        links = get_playlist_video_urls(YOUTUBE_URL)
        for link in links:
            re_ydl_opts_resolution(link,resolution,out_path)
            yield from download_Thread(ydl_opts,link)
        if has_files('Downloads/Video'):
            yield "影片下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取影片"

def download_audio(YOUTUBE_URL, do_zip):
    if(do_zip):
        ydl_opts = {
        'format': 'bestaudio/best',  # 下載最佳畫質
        'outtmpl': 'Downloads_TMP/%(title)s.%(ext)s',  # 影片文件名格式
        'postprocessors': [{  # 下載後處理選項
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        },{
            'key': 'FFmpegMetadata'
        }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads_TMP'):
            zip_files('Audio')
            yield "音檔下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取音檔"
    else:
        # 音檔下載
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'Downloads/Audio/%(title)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            },{
            'key': 'FFmpegMetadata'
        }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads/Audio'):
            yield "音檔下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取音檔"
            
    
def has_files(folder_path):
    for _, _, files in os.walk(folder_path):
        if files:
            return True
    return False

def zip_files(Type):
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute
    second = now.second
    folder_to_zip = 'Downloads_TMP'
    if has_files(folder_to_zip):
        zip_folder = 'Downloads_ZIP'
        if not os.path.exists(zip_folder):
            os.makedirs(zip_folder)
        
        zip_filename = f'{zip_folder}/{Type}_{year}_{month}_{day}_{hour}_{minute}_{second}.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, dirs, files in os.walk(folder_to_zip):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))
        # 檢查資料夾是否存在
        folder_to_zip = 'Downloads_TMP'
        if os.path.exists(folder_to_zip):
            shutil.rmtree(folder_to_zip)


def gradio_interface(function_choice, YOUTUBE_URL, do_zip,quality,resolution):
    global resolution_option
    global resolution_high
    global resolution_avg
    global resolution_low
    global playlist_info
    global count
    # 全局變量或全局狀態，用於保存播放清單信息
    playlist_info = {
        'total_videos': 0,
        'downloaded_videos': 0,
    }
    get_video_quantity(YOUTUBE_URL)
    count = True
    # 檢查資料夾是否存在
    folder_to_zip = 'Downloads_TMP'
    if os.path.exists(folder_to_zip):
        shutil.rmtree(folder_to_zip)
    if quality == "最高":
       resolution_option = resolution_high
    elif quality == "平均":
        resolution_option = resolution_avg
    elif quality == "最低":
        resolution_option = resolution_low
    if function_choice == "下載影片":
        yield from download_video(YOUTUBE_URL, do_zip,resolution_option[resolution])
    elif function_choice == "下載音檔":
        yield from download_audio(YOUTUBE_URL, do_zip)

def update_quality(function_choice):
    if function_choice == "下載影片":
        return gr.update(visible=True), gr.update(visible=True)  # 影片功能顯示質量和解析度選項
    else:
        return gr.update(visible=False), gr.update(visible=False)  # 音檔功能隱藏質量和解析度選項
    

# 使用 Gradio Blocks 來建立介面
with gr.Blocks() as iface:
    function_choice = gr.Radio(["下載影片", "下載音檔"], label="選擇功能")
    YOUTUBE_URL = gr.Textbox(label="輸入 YouTube 清單或影片的連結")
    do_zip = gr.Checkbox(label="壓縮成 ZIP 檔")
    quality = gr.Radio(["最高","平均",'最低'], label="選擇畫質(畫質越高，檔案越大)", visible=False)
    resolution = gr.Dropdown(list(resolution_option.keys()), label="選擇解析度", visible=False)
    
    function_choice.change(fn=update_quality, inputs=function_choice, outputs=[quality, resolution])
    
    iface_interface = gr.Interface(
        fn=gradio_interface,
        inputs=[
            function_choice,
            YOUTUBE_URL,
            do_zip,
            quality,
            resolution
        ],
        outputs="text"
    )

# 個人電腦可以改成 share=False ， Colab必須要是 share=True
iface.queue().launch(share=True)
# 4K 60fps 影片連結(測試用) : https://youtu.be/3aRJtJRa434
# 播放清單 和 影片 不可以放分享連結(share結尾)，要放一般的網址，如 : https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL

/home/tsukisama9292/公共/Ubuntu-Useful-Guide/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://adeb0866d5b5c2f59b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL: Downloading webpage
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Test_dowload
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


...

[youtube:tab] Playlist Test_dowload: Downloading 2 items of 2
[download] Downloading item 1 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=3aRJtJRa434
[youtube] 3aRJtJRa434: Downloading webpage
[youtube] 3aRJtJRa434: Downloading ios player API JSON
[youtube] 3aRJtJRa434: Downloading m3u8 information
[download] Downloading item 2 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=b-51DkVWxss
[youtube] b-51DkVWxss: Downloading webpage
[youtube] b-51DkVWxss: Downloading ios player API JSON
[youtube] b-51DkVWxss: Downloading m3u8 information
[download] Finished downloading playlist: Test_dowload
[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL: Downloading webpage
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Test_dowload
[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL

[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


[youtube:tab] PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL page 1: Downloading API JSON


[youtube:tab] Playlist Test_dowload: Downloading 2 items of 2
[download] Downloading item 1 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=3aRJtJRa434
[youtube] 3aRJtJRa434: Downloading webpage
[youtube] 3aRJtJRa434: Downloading ios player API JSON
[youtube] 3aRJtJRa434: Downloading m3u8 information
[info] 3aRJtJRa434: Downloading 1 format(s): 251
[download] Destination: Downloads/Audio/【4K⧸60fps⧸MMD】  恋愛裁判 ⧸ Love Trial — YYB 初音MIKU.webm
[download] 100% of    2.66MiB in 00:00:00 at 10.33MiB/s  
[ExtractAudio] Destination: Downloads/Audio/【4K⧸60fps⧸MMD】  恋愛裁判 ⧸ Love Trial — YYB 初音MIKU.mp3
Deleting original file Downloads/Audio/【4K⧸60fps⧸MMD】  恋愛裁判 ⧸ Love Trial — YYB 初音MIKU.webm (pass -k to keep)
[Metadata] Adding metadata to "Downloads/Audio/【4K⧸60fps⧸MMD】  恋愛裁判 ⧸ Love Trial — YYB 初音MIKU.mp3"
[download] Downloading item 2 of 2
[youtube] Extracting URL: https://www.youtube.com/watch?v=b-51DkVWxss
[youtube] b-51DkVWxss: Downloading webpage
[youtube] b-51DkVWxss: Downloadin

In [ ]:
from datetime import datetime
import yt_dlp
# 原程式開源者:Github.com/TsukiSama9292
def get_current_time():
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M")

def get_video_info(youtube_url):
    ydl = yt_dlp.YoutubeDL()
    info = ydl.extract_info(youtube_url, download=False)
    title = info.get('title', '未知標題')
    uploader = info.get('uploader', '未知')
    upload_date = info.get('upload_date', '未知')
    
    if upload_date != '未知':
        formatted_upload_date = f"{upload_date[:4]}-{upload_date[4:6]}-{upload_date[6:]}"
    else:
        formatted_upload_date = '未知'
    
    return title, uploader, formatted_upload_date

def prepare_ydl_opts(out_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'verbose': True,
        'outtmpl': out_path,
        'merge-output-format': 'mkv',
        'embed-metadata': True,
        'parse-metadata': '%(artist)s - %(title)s',
        'postprocessors': [
            {
                'key': 'FFmpegMetadata'
            }
        ]
    }
    
    return ydl_opts

# 示例用法
youtube_url = 'https://youtu.be/3aRJtJRa434'
out_path = 'Downloads/%(title)s.%(ext)s'
ydl_opts = prepare_ydl_opts(youtube_url, out_path)

# 下載影片
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])
